In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

In [2]:
# 드라이버 불러오기
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

url = 'https://www.thecocktailproject.com'
driver.get(url) # go to website
time.sleep(1)

# 쿠키 및 정보 입력

In [3]:
# Occasions list 정리하기
occasions = ['bbq','birthday','ladies-night','brunch','christmas','fall','game-day','halloween',
 'holiday','new-years','spring','summer','thanksgiving','tropical','valentines-day','wedding','winter']

In [4]:
data = []

for occasion in occasions:
    url = 'https://www.thecocktailproject.com/' + occasion + '-cocktail-recipes'
    driver.get(url) # go to website
    time.sleep(3)

    cocktails = [] 
    index = 0

    while True:
        # Scrape elements in the page
        elements = driver.find_elements(By.CSS_SELECTOR,"div.col-12.col-sm-6.col-md-6.col-lg-4.col-xl-4")

        if elements:
            for element in elements:
                info = element.find_element(By.CLASS_NAME,'item-desc')
                name = info.find_element(By.TAG_NAME,'a').text.strip() # 칵테일 이름 긁어오기
                cocktails.append(name)
                
            index += 1
            url = 'https://www.thecocktailproject.com/' + occasion + '-cocktail-recipes/?page=' + str(index)
            driver.get(url) # go to website
            time.sleep(3)
        else:
            break
    
    data.append(cocktails)
    
    print(occasion,': ',len(cocktails))



bbq :  127
birthday :  78
ladies-night :  84
brunch :  52
christmas :  42
fall :  151
game-day :  27
halloween :  15
holiday :  100
new-years :  19
spring :  113
summer :  253
thanksgiving :  54
tropical :  69
valentines-day :  22
wedding :  19
winter :  68


In [5]:
data_csv = {'Occasion': occasions,
        'Cocktails': data}

df = pd.DataFrame(data_csv)
df

,Occasion,Cocktails
0,bbq,"[Tom Collins, Tokyo Collins, Gin Basil Smash, ..."
1,birthday,"[Espresso Martini, French 75, Gimlet, 'Tea' Ne..."
2,ladies-night,"[EFFEN® Blood Orange Mimosa, Cucumber Watermel..."
3,brunch,"[Tom Collins, Tokyo Collins, Gin Basil Smash, ..."
4,christmas,"[Jim Beam® and Ginger Highball, Hornitos® Palo..."
5,fall,"[Espresso Martini, Japanese Gin Sour, Vesper, ..."
6,game-day,"[SQRRL® PB&J Shot, DeKuyper® End of the Rainbo..."
7,halloween,"[Cruzan®\nSpiced Apple Cider, Cruzan® Cider Pu..."
8,holiday,"[French 75, Jim Beam® and Ginger Highball, 'Te..."
9,new-years,"[Espresso Martini, French 75, Blue Flurry, Kis..."


In [7]:
df.to_csv("cocktail_occasion.csv") # export to csv file

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)

# Wait for the page to load
time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

#### 여기까지 access

# Find all cocktail elements
cocktail_elements = driver.find_elements(By.CSS_SELECTOR, '.receipe-grid-items')

# Initialize an empty dictionary to store cocktail names and links
cocktails_dict = {}

# Loop through each cocktail element
for cocktail_element in cocktail_elements:
    # Extract name of the cocktail
    name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()
    
    # Extract link of the cocktail
    link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')
    
    driver.get(link)

    ingredient_elements = driver.find_elements(By.CSS_SELECTOR, '.ingredients-block')
    ingredients = [ingredient.text.strip() for ingredient in ingredient_elements]

    spirit_elements = driver.find_elements(By.CSS_SELECTOR, 'div.drink-properties-incredible')
    for spirit in spirit_elements:
        spirits = [print(sp.text.strip())  for sp in spirit_elements[0].find_element(By.TAG_NAME,'p').find_elements(By.TAG_NAME,'a')]



    rating_count_elements = driver.find_elements(By.CSS_SELECTOR, '.item-list ul li')
    ratings_count = [rating_count.text.strip() for rating_count in rating_count_elements]



    # Add the name and link of the cocktail to the dictionary
    cocktails_dict[name] = {'link': link, 'ingredients': ingredients, 'main_spirit': spirits, 'ratings_count': ratings_count}

    driver.back()

len(cocktails_dict)
cocktails_dict







Gin
Gin
Gin
Gin
Gin
Gin
Gin
Gin
Gin
Gin
Gin
Gin


KeyboardInterrupt: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Create an empty dictionary to store ingredients for each cocktail
cocktails_with_description = {}

for name, data in cocktails_dict.items():
    link = data['link']
    driver.get(link)

    try:
    # Find the description element
        description_element = driver.find_element_by_css_selector('.recipe-copy.desktop-only p')
        # Extract the text of the description
        description = description_element.text.strip()

    except Exception as e:
        description = "NA"

    # Assign the description to the current cocktail
    data['description'] = description
    
    # Add the updated cocktail data to the new dictionary
    cocktails_with_description[name] = data

# Close the browser
driver.quit()

# Update cocktails_dict with the new dictionary containing ingredient information
cocktails_dict.update(cocktails_with_description)